In [10]:
import pandas as pd
import numpy as np
import holidays
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats

In [11]:
def prepare_hourly_data(df: pd.DataFrame) -> pd.DataFrame:
    # Configurar índice temporal para la descomposición
    df = df.copy()
    
    # Eliminar datos duplicados
    df = df.drop_duplicates(subset='Fecha')
    
    # Setear la fecha en el indice    
    df = df.set_index('Fecha').asfreq('h')
    
    # Interpolar valores faltantes
    df['Volumen'] = df['Volumen'].interpolate()
    
    return df

In [12]:
def extract_datetime_features(df: pd.DataFrame, fecha_col='Fecha') -> pd.DataFrame:
    # Copia para no modificar el original
    df_result = df.copy()
    
    if fecha_col not in df_result.columns:
        df_result[fecha_col] = df_result.index
    
    # Características básicas de tiempo
    df_result['hora'] = df_result[fecha_col].dt.hour
    df_result['dia'] = df_result[fecha_col].dt.day
    df_result['dia_semana'] = df_result[fecha_col].dt.dayofweek 
    df_result['dia_year'] = df_result[fecha_col].dt.dayofyear
    df_result['semana_year'] = df_result[fecha_col].dt.isocalendar().week
    df_result['mes'] = df_result[fecha_col].dt.month
    df_result['trimestre'] = df_result[fecha_col].dt.quarter
    df_result['year'] = df_result[fecha_col].dt.year
    
    # Características derivadas
    df_result['fin_de_semana'] = df_result['dia_semana'].isin([5, 6]).astype(int)
    df_result['dia_laboral'] = (~df_result['dia_semana'].isin([5, 6])).astype(int)
    
    # Festivos en Colombia
    festivos_colombia = holidays.country_holidays('CO', years=df_result[fecha_col].dt.year.unique())
    df_result['es_festivo'] = df_result[fecha_col].dt.date.isin(
        [d for d in festivos_colombia]
    ).astype(int)
        
    # Características cíclicas (seno y coseno)
    df_result['hora_seno'] = np.sin(2 * np.pi * df_result['hora'] / 24)
    df_result['hora_coseno'] = np.cos(2 * np.pi * df_result['hora'] / 24)
    df_result['dia_semana_seno'] = np.sin(2 * np.pi * df_result['dia_semana'] / 7)
    df_result['dia_semana_coseno'] = np.cos(2 * np.pi * df_result['dia_semana'] / 7)
    df_result['mes_seno'] = np.sin(2 * np.pi * df_result['mes'] / 12)
    df_result['mes_coseno'] = np.cos(2 * np.pi * df_result['mes'] / 12)
    
    return df_result  

In [13]:
def extract_seasonal_features(df: pd.DataFrame, period=24):
    # Crear copia para no modificar el original
    df_result = df.copy()
    
    if 'Fecha' not in df_result.columns:
        df_result['Fecha'] = df_result.index

    # Verificar frecuencia horaria
    if not isinstance(df_result.index, pd.DatetimeIndex):
        df_result = df_result.set_index('Fecha').asfreq('h')
    
    # Realizar descomposición estacional
    result = seasonal_decompose(df_result['Volumen'], model='additive', period=period)
    
    # Añadir componentes al DataFrame original
    df_result['trend'] = result.trend.values
    df_result['seasonal'] = result.seasonal.values
    df_result['residual'] = result.resid.values
    
    # Calcular características adicionales
    df_result['detrended'] = df_result['Volumen'] - df_result['trend']
    df_result['seas_strength'] = abs(df_result['seasonal'] / df_result['Volumen'])
    df_result['seas_norm'] = df_result['seasonal'] / df_result['seasonal'].std()
    
    return df_result

In [14]:
def calculate_rolling_features(df: pd.DataFrame, target_col='Volumen', windows=[24, 48, 168]):
    
    # Crear copia para no modificar el original
    df_result = df.copy()
    
    # Calcular estadísticas para cada ventana
    for window in windows:
        
        # Estadísticas móviles
        df_result[f'rolling_mean_{window}h'] = df_result[target_col].rolling(window=window, min_periods=1).mean()
        df_result[f'rolling_std_{window}h'] = df_result[target_col].rolling(window=window, min_periods=1).std()
        df_result[f'rolling_min_{window}h'] = df_result[target_col].rolling(window=window, min_periods=1).min()
        df_result[f'rolling_max_{window}h'] = df_result[target_col].rolling(window=window, min_periods=1).max()
        
        # Diferencias con respecto al promedio móvil
        df_result[f'diff_from_mean_{window}h'] = df_result[target_col] - df_result[f'rolling_mean_{window}h']
        df_result[f'pct_diff_from_mean_{window}h'] = (df_result[target_col] / df_result[f'rolling_mean_{window}h'] - 1) * 100
        
    # Lags
    for i in range(1,24):
        df_result[f'lag_{i}'] = df_result[target_col].shift(i)
        
    # Patrones semanales (168 horas)
    if 168 in windows:
        df_result['diff_from_last_week'] = df_result[target_col].diff(168)
        df_result['pct_diff_from_last_week'] = df_result[target_col].pct_change(168) * 100
    
    return df_result

In [15]:
def detect_outliers(df: pd.DataFrame, columns, method='zscore', threshold=3.0):
    
    # Crear copia para no modificar el original
    df_result = df.copy()
    
    for col in columns:
        if col not in df_result.columns:
            print(f"Columna {col} no encontrada en el DataFrame")
            continue
            
        if method == 'zscore':
            # Método Z-score
            z_scores = stats.zscore(df_result[col], nan_policy='omit')
            df_result[f'{col}_outlier_zscore'] = (abs(z_scores) > threshold).astype(int)
            
        elif method == 'iqr':
            # Método IQR
            Q1 = df_result[col].quantile(0.25)
            Q3 = df_result[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            df_result[f'{col}_outlier_iqr'] = ((df_result[col] < lower_bound) | 
                                               (df_result[col] > upper_bound)).astype(int)
    
    # Columna agregada de outliers
    outlier_cols = [col for col in df_result.columns if '_outlier_' in col]
    if outlier_cols:
        df_result['is_any_outlier'] = df_result[outlier_cols].max(axis=1)
    
    return df_result

In [16]:
def create_features_for_anomaly_detection(df: pd.DataFrame):
    """Función principal para crear todas las características para detección de anomalías"""
    # 0. Preparar los datos
    df_processed = prepare_hourly_data(df)
    
    # 1. Extraer características temporales
    df_processed = extract_datetime_features(df_processed)
    
    # 2. Añadir características de estacionalidad
    df_processed = extract_seasonal_features(df_processed)
    
    # 3. Calcular promedios móviles y estadísticas relacionadas
    df_processed = calculate_rolling_features(df_processed, target_col='Volumen')
            
    # 4. Detectar outliers en columnas relevantes
    columns_for_outlier_detection = ['Volumen']
    
    # Detectar outliers con ambos métodos
    df_processed = detect_outliers(df_processed, columns_for_outlier_detection, method='zscore')
    df_processed = detect_outliers(df_processed, columns_for_outlier_detection, method='iqr')
    
    return df_processed

In [17]:
# Código principal para procesar datos
def process_gas_data(data: pd.DataFrame):
    """Procesa datos de gas por cliente para detección de anomalías"""
    results = {}
    
    for cliente, df_cliente in data.groupby('Cliente'):
        print(f"\nProcesando cliente: {cliente}")
        
        # Aplicar todas las funciones de ingeniería de características
        df_processed = create_features_for_anomaly_detection(df_cliente)
        
        # Guardar resultados
        results[cliente] = df_processed
        
        # Mostrar resumen de outliers detectados
        n_outliers = df_processed['is_any_outlier'].sum()
        print(f"Se detectaron {n_outliers} [{n_outliers/len(df_processed) * 100:2f}%] posibles anomalías para el cliente {cliente}")
        
        # Guardar en csv
        df_processed.to_csv(f"../data/processed/{cliente}.csv", index=False)
    
    return results

In [18]:
# Cargar datos
data = pd.read_csv("../data/raw/data.csv", parse_dates=['Fecha'])

# Procesar datos
results = process_gas_data(data)


Procesando cliente: CLIENTE1
Se detectaron 4 [0.009193%] posibles anomalías para el cliente CLIENTE1

Procesando cliente: CLIENTE10
Se detectaron 5804 [13.338849%] posibles anomalías para el cliente CLIENTE10

Procesando cliente: CLIENTE11
Se detectaron 146 [0.335540%] posibles anomalías para el cliente CLIENTE11

Procesando cliente: CLIENTE12
Se detectaron 7 [0.016088%] posibles anomalías para el cliente CLIENTE12

Procesando cliente: CLIENTE13
Se detectaron 6262 [14.391432%] posibles anomalías para el cliente CLIENTE13

Procesando cliente: CLIENTE14
Se detectaron 123 [0.282681%] posibles anomalías para el cliente CLIENTE14

Procesando cliente: CLIENTE15
Se detectaron 5705 [13.111326%] posibles anomalías para el cliente CLIENTE15

Procesando cliente: CLIENTE16
Se detectaron 1 [0.002298%] posibles anomalías para el cliente CLIENTE16

Procesando cliente: CLIENTE17
Se detectaron 198 [0.455047%] posibles anomalías para el cliente CLIENTE17

Procesando cliente: CLIENTE18
Se detectaron 533